In [ ]:
"""
Author: Sixteen01 Capital
Version: 3.1.0 (Multi-exchange support)
Date: 08/08/2025

Function: Monitor combined equity every 15 minutes and save it into database.
Structure:
    - equity_log(timestamp, bybit001, deribit001, total)
"""

import pandas as pd
import sqlite3
import schedule
import time
from datetime import datetime, timezone
import ccxt
import json

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', 500)

# To Load the account from Multi Exchange
def load_selected_accounts(account_names, json_file="accounts.json"):
    with open(json_file) as file:
        all_accounts = json.load(file)

    loaded_accounts = []
    for name in account_names:
        account_info = all_accounts.get(name)
        if account_info:
            exchange_name = account_info.get("exchange")
            if exchange_name:
                try:
                    exchange_class = getattr(ccxt, exchange_name)
                    account_object = exchange_class({
                        "apiKey": account_info["apiKey"],
                        "secret": account_info["secret"]
                    })
                    loaded_accounts.append((account_object, name, exchange_name))
                except Exception as e:
                    print(f"⚠️ Error loading {name}: {e}")
    return loaded_accounts


# ====== Select accounts
selected_accounts = ["bybit001", "deribit001"]
accounts = load_selected_accounts(selected_accounts)

# ====== Initialize DB
def init_db():
    with sqlite3.connect("equity_monitor.db") as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS equity_log (
                timestamp INTEGER PRIMARY KEY
            )
        """)

# ====== Save to DB
def save_combined_to_db(timestamp, equity_dict):
    total = sum(equity_dict.values())
    columns = ['timestamp'] + list(equity_dict.keys()) + ['total']
    values = [timestamp] + list(equity_dict.values()) + [total]

    with sqlite3.connect("equity_monitor.db") as conn:
        cursor = conn.cursor()

        # Get existing columns
        cursor.execute("PRAGMA table_info(equity_log)")
        existing_cols = [row[1] for row in cursor.fetchall()]

        # Add missing columns
        for col in equity_dict.keys():
            if col not in existing_cols:
                cursor.execute(f"ALTER TABLE equity_log ADD COLUMN {col} REAL")
        if 'total' not in existing_cols:
            cursor.execute("ALTER TABLE equity_log ADD COLUMN total REAL")

        # Insert row
        insert_sql = f"INSERT OR REPLACE INTO equity_log ({', '.join(columns)}) VALUES ({', '.join(['?'] * len(columns))})"
        cursor.execute(insert_sql, values)

# ====== Balance fetcher
def get_balance(account_tuple, retries=3, delay=5):
    account, name, exchange = account_tuple

    for attempt in range(1, retries + 1):
        try:
            if exchange == "bybit":
                data = account.privateGetV5AccountWalletBalance({"accountType": "UNIFIED"})
                equity = float(data["result"]["list"][0]["totalEquity"])
                return equity
            elif exchange == "deribit":
                data = account.fetch_balance()
                equity = float(data["total"].get("USDT", 0))  # or use "BTC"
                return equity
        except Exception as e:
            print(f"[Attempt {attempt}] Error fetching {name} ({exchange}): {e}")
            time.sleep(delay)
    print(f"❌ Failed to fetch balance for {name}")
    return None

# ====== Run full equity check
def get_all_balance():
    minute_floor = datetime.now(timezone.utc).replace(second=0, microsecond=0)
    timestamp = int(minute_floor.timestamp() * 1000)
    utc_time_str = minute_floor.strftime("%Y-%m-%d %H:%M:%S")

    equity_dict = {}
    for ac in accounts:
        account_name = ac[1]
        balance = get_balance(ac)
        if balance is None:
            print(f"[{utc_time_str}] Failed to get balance for {account_name}. Aborting.")
            return
        equity_dict[account_name] = balance
        print(f"[{utc_time_str}] {account_name}: {balance:.2f}")

    save_combined_to_db(timestamp, equity_dict)
    print(f"[{utc_time_str}] ✅ Data saved at timestamp {timestamp}")

# ====== Main loop
if __name__ == '__main__':
    init_db()
    get_all_balance()

    schedule.every().hour.at(":00").do(get_all_balance)
    schedule.every().hour.at(":15").do(get_all_balance)
    schedule.every().hour.at(":30").do(get_all_balance)
    schedule.every().hour.at(":45").do(get_all_balance)

    while True:
        schedule.run_pending()
        time.sleep(10)
